In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install streamlit
!pip install transformers
!pip install datasets nltk
!pip install datasets --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 92.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behavio

In [ ]:
!pip install evaluate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.4 MB/s eta 0:00:00


In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=6b14f9663e208dee9c0e8130163cd5204429380436c7a7309bb065e83361f91f
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
%%writefile app.py
import streamlit as st
from transformers import T5Tokenizer, T5ForConditionalGeneration
import re

# Load your fine-tuned T5 model and tokenizer
@st.cache_resource
def load_model():
    model = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/summarization_model")  # Replace with your model path
    tokenizer = T5Tokenizer.from_pretrained("t5-small")
    return model, tokenizer

@st.cache_resource
def load_model2():
    model2 = T5ForConditionalGeneration.from_pretrained("/content/drive/MyDrive/model2")  # Replace with your model path
    tokenizer2 = T5Tokenizer.from_pretrained("t5-small")
    return model2, tokenizer2

def preprocess_article(raw_text):
    text = raw_text.strip()
    # Normalize whitespace and remove URLs
    text = re.sub(r"\s+", " ", re.sub(r"http\S+|www\S+|https\S+", "", text))
    # Use semantic tags for context (optional but can help models focus)
    return f"<summary> {text} </summary>"

def chunk_text_with_overlap(text, tokenizer, chunk_size=400, overlap=50):
    tokens = tokenizer.encode("summarize: " + text, return_tensors="pt")[0]
    chunks = []
    for i in range(0, len(tokens), chunk_size - overlap):
        chunk = tokens[i:i + chunk_size]
        chunks.append(chunk)
        if i + chunk_size >= len(tokens):  # Avoid breaking at the last part
            break
    return chunks

def summarize_long_text(text, model, tokenizer, chunk_size=400):
    chunks = chunk_text_with_overlap(text, tokenizer, chunk_size=chunk_size)
    summaries = []

    for chunk in chunks:
        chunk = chunk.unsqueeze(0)  # Add batch dimension
        summary_ids = model.generate(
            chunk,
            max_length=200,  # Ensure summaries are concise
            min_length=100,  # Encourage longer summaries than default
            num_beams=8,  # Increase diversity
            top_p=0.9,  # Add controlled randomness
            temperature=0.8,  # Adjust temperature for coherent variety
            length_penalty=1.2,  # Penalize overly short outputs
            early_stopping=True  # Stop generation early for more precise outputs
        )
        summaries.append(tokenizer.decode(summary_ids[0], skip_special_tokens=True))

    # Post-process to clean and merge summaries
    final_summary = " ".join(summaries).strip()
    final_summary = re.sub(r"\s+", " ", final_summary)  # Normalize spaces
    final_summary = final_summary.replace("<summary>", "").replace("</summary>", "")
    return final_summary
def preprocess_article_with_context(raw_text):
    text = raw_text.strip()
    text = re.sub(r"\s+", " ", re.sub(r"http\S+|www\S+|https\S+", "", text))
    return f"<summary> {text} </summary>"  # Add semantic tags
def remove_repeated_sentences(text):
    sentences = text.split(". ")
    seen = set()
    result = []
    for sentence in sentences:
        if sentence not in seen:
            result.append(sentence)
            seen.add(sentence)
    return ". ".join(result)
def clean_summary(summary):
    # Remove unwanted punctuation artifacts
    summary = re.sub(r"^[,.\s]+", "", summary)  # Remove leading punctuation
    summary = re.sub(r"[,.\s]+$", "", summary)  # Remove trailing punctuation
    summary = re.sub(r"\s+", " ", summary).strip()  # Normalize whitespace
    return summary
def summarize_model2(text, model, tokenizer):
    # Preprocess input text
    text = preprocess_article_with_context(text)

    # Tokenize input
    inputs = tokenizer(text, return_tensors="pt", max_length=1024, truncation=True)

    # Generate summary with tuned parameters
    outputs = model.generate(
        inputs.input_ids,
        max_length=150,
        min_length=50,
        num_beams=6,
        no_repeat_ngram_size=3,
        length_penalty=1.5,
        early_stopping=True,
        temperature=0.7,
        top_p=0.95
    )

    # Decode and clean output
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    summary = remove_repeated_sentences(summary)
    summary = clean_summary(summary)
    return summary




# Streamlit App
st.title("Text Summarizer")
st.subheader("Enter a article to find its summary.")

# Input text area
input_text = st.text_area("Enter the article to summarize here", height=300)
uploaded_file = st.file_uploader("Or upload a text file", type="txt")

if uploaded_file is not None:
    input_text = uploaded_file.read().decode("utf-8")  # Decode file content to string


# Load model and tokenizer
model, tokenizer = load_model()
# Load model and tokenizer
model2, tokenizer2 = load_model2()
# Summarize button
# Dropdown for selecting summary type
summary_type = st.selectbox("Select the type of summary:", ["Short summary", "Long summary"])

if summary_type == "Short summary":
    if input_text.strip():
        st.write("Short summary: ")
        try:
            # Preprocess and summarize
            processed_text = preprocess_article(input_text)
            summary = summarize_long_text(processed_text, model, tokenizer)
            summary = summary.replace("title>", "")
            summary = summary.replace("/summary>", "")
            summary = remove_repeated_sentences(summary)
            st.write(summary)
        except Exception as e:
            st.error(f"An error occurred: {e}")
    else:
        st.warning("Please enter some text to summarize.")

elif summary_type == "Long summary":
    if input_text.strip():
        st.write("Long summary: ")
        try:
            # Preprocess and summarize
            summary = summarize_model2(input_text, model2, tokenizer2)
            summary = summary.replace("/summary>", "")
            st.write(summary)
        except Exception as e:
            st.error(f"An error occurred: {e}")
    else:
        st.warning("Please enter some text to summarize.")



Writing app.py


In [ ]:
!ngrok authtoken 2pyyrJpCg7XgQ3r43cdYdRVb9xk_2SVnaawhzpbsqU1iXyPDW

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
from pyngrok import ngrok
!streamlit run app.py &>/dev/null&
public_url = ngrok.connect(addr='8501')
print(f"Streamlit app is live at {public_url}")

Streamlit app is live at NgrokTunnel: "https://cf68-34-125-56-189.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
!pip install streamlit pyngrok

In [ ]:
text = "This is the first sentence. This is the second sentence. This is the third sentence."

# Split the text into sentences
sentences = text.split(". ")

# Tagging the first sentence with <first> and the rest with <body>
tagged_text = "<first> " + sentences[0] +". "
tagged_text += "<body> " + ". ".join(sentences[1:]) + " "

# Output the tagged text
print(tagged_text)

<first> This is the first sentence. <body> This is the second sentence. This is the third sentence. 
